In [1]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import zipfile
import os
#taking zip file name
zip_file = "archive (1).zip" 
extract_folder = "dataset"
if not os.path.exists(extract_folder):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("ZIP file  is extracted successfully")
else:
    print("Dataset already extracted")
    
# step1: Import Required Libraries
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# step2: Feature Extraction Function

def extract_features(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None

    image = cv2.resize(image, (128, 128))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    features = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )
    return features
#step3: Load Dataset

data = []
labels = []

base_path = "dataset/animals"
cat_path = os.path.join(base_path, "cat")
dog_path = os.path.join(base_path, "dog")

# Safety check in done here
print("Cat folder exists:", os.path.exists(cat_path))
print("Dog folder exists:", os.path.exists(dog_path))

# We Load Cat Images
for img in os.listdir(cat_path):
    img_path = os.path.join(cat_path, img)
    features = extract_features(img_path)
    if features is not None:
        data.append(features)
        labels.append(0)   

#  We Load Dog Images
for img in os.listdir(dog_path):
    img_path = os.path.join(dog_path, img)
    features = extract_features(img_path)
    if features is not None:
        data.append(features)
        labels.append(1)   

data = np.array(data)
labels = np.array(labels)

print("Total samples:", len(data))

# step4: Train-Test Split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

#step5: Train SVM Model
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train, y_train)

# step6: Model Evaluation

y_pred = svm.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=["Cat", "Dog"]))


# step7: Predict New Image

def predict_image(image_path):
    features = extract_features(image_path)
    if features is None:
        print("Invalid image")
        return

    prediction = svm.predict([features])

    if prediction[0] == 0:
        print("Prediction: Cat 🐱")
    else:
        print("Prediction: Dog 🐶")

Dataset already extracted
Cat folder exists: True
Dog folder exists: True
Total samples: 1000
Accuracy: 0.91

Classification Report:

              precision    recall  f1-score   support

         Cat       0.87      0.96      0.91        96
         Dog       0.96      0.87      0.91       104

    accuracy                           0.91       200
   macro avg       0.91      0.91      0.91       200
weighted avg       0.91      0.91      0.91       200

